In [33]:
# !pip install urllib3==1.21.1
# !pip install webdriver-manager
# !pip install selenium

In [3]:
def bloomberg_scrap(keyword):
    import webbrowser
    import requests
    from bs4 import BeautifulSoup
    import os
    import re
    import time
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    options = Options()
    options.headless = True
    chrome_path=r"C:\Users\ADMIN\Desktop\CIMB Work\chromedriver.exe"
    from datetime import datetime, timedelta
    todaysdate=datetime.now().strftime("%Y-%m-%d")
    last_date=(datetime.now() + timedelta(days=-6)).strftime("%Y-%m-%d")
    url_base=[]
    headline=[]
    headline_url=[]
    cat=[]

    for key in keyword:
        base_url="https://www.bloomberg.com/search?query={}".format(key)
        driver=webdriver.Chrome(chrome_path)
        driver.get(base_url)
        time.sleep(3)
        real_soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()
        ab=real_soup.find_all("div", {"class": "storyItem__4d5aa17d67"})
        if(ab!=None):
            for i in ab:
                a=i.find("a",href=True)
                head_url=a.get("href")
                headline_url.append(head_url)
                b=i.find("a",class_='headline__96ba1917df')
                head_title=b.get_text().strip()
                headline.append(head_title)
                url_base.append(base_url)
                cat.append(key)
        else:
            headline_url.append("No url found")
            headline.append("No headline found")
            url_base.append(base_url)
            cat.append(key)
            
    import pandas as pd
    df = pd.DataFrame({'keyword':cat,'main_url':url_base, 'Headline':headline,'headline_url': headline_url })
    df.drop_duplicates(inplace=True)
    df["source"]="Bloomberg"       
    
    from selenium import webdriver
    #object of ChromeOptions class
    c = webdriver.ChromeOptions()
    #incognito parameter passed
    c.add_argument("--incognito")
    import re
    list=[]
    for ind in df.index:
        dict={}
        try:
            driver = webdriver.Chrome(chrome_path,options=c)
            driver.implicitly_wait(0.5)
            driver.get(df['headline_url'][ind])
            time.sleep(5)
            real_soup = BeautifulSoup(driver.page_source, 'html.parser')
            driver.quit()
            article=[]
            ab=real_soup.find_all("div", {"class":"body-content fence-body"})
            summary=[]
            for i in ab:
                summary.append(re.sub(r'\s+',' ',i.text.strip()))   
                fn_out=' '.join([str(i) for i in summary])
            dt=str(real_soup.find("time").text.strip().replace("@",""))
            dict['Keyword']=df['keyword'][ind]
            dict['Date']=dt
            dict['main_url']=df['main_url'][ind]
            dict['Headline']=df['Headline'][ind]
            dict['headline_url']=df['headline_url'][ind]
            dict['Article']=fn_out
            dict['Source']=df['source'][ind]

            list.append(dict)

        except:
            print("*"*20,"error","*"*20)
            pass
    news_df=pd.DataFrame(list)


    return news_df 

In [4]:
df_Bloomberg=bloomberg_scrap(keyword)

<ipython-input-3-8bc6eead5232>:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(chrome_path)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
df_Bloomberg.head(10)

,keyword,website,url,date_time,title,summary,article
0,RP TRADING,Bloomberg,https://www.bloomberg.com/news/articles/2021-1...,2021-11-18,Are you a robot?,Why did this happen?\nPlease make sure your br...,Why did this happen?\n\nPlease make sure your ...
1,RP TRADING,Bloomberg,https://www.bloomberg.com/view/articles/2020-0...,2020-01-02,Are you a robot?,Why did this happen?\nPlease make sure your br...,Why did this happen?\n\nPlease make sure your ...
2,RP TRADING,Bloomberg,https://www.bloomberg.com/news/articles/2013-1...,2013-12-04,Are you a robot?,Why did this happen?\nPlease make sure your br...,Why did this happen?\n\nPlease make sure your ...
3,RP TRADING,Bloomberg,https://www.bloomberg.com/news/articles/2013-1...,2013-12-17,Are you a robot?,Why did this happen?\nPlease make sure your br...,Why did this happen?\n\nPlease make sure your ...
4,RP TRADING,Bloomberg,https://www.bloomberg.com/news/articles/2011-0...,2011-04-07,Are you a robot?,Why did this happen?\nPlease make sure your br...,Why did this happen?\n\nPlease make sure your ...
5,RP TRADING,Bloomberg,https://www.bloomberg.com/news/articles/2010-0...,2010-09-02,Are you a robot?,Why did this happen?\nPlease make sure your br...,Why did this happen?\n\nPlease make sure your ...
6,RP TRADING,Bloomberg,https://www.bloomberg.com/news/articles/2010-0...,2010-07-02,Are you a robot?,Why did this happen?\nPlease make sure your br...,Why did this happen?\n\nPlease make sure your ...
7,RP TRADING,Bloomberg,https://www.bloomberg.com/news/articles/2007-0...,2007-07-13,Are you a robot?,Why did this happen?\nPlease make sure your br...,Why did this happen?\n\nPlease make sure your ...
8,RP TRADING,Bloomberg,https://www.bloomberg.com/view/articles/2020-0...,2020-04-07,Are you a robot?,Why did this happen?\nPlease make sure your br...,Why did this happen?\n\nPlease make sure your ...
9,RP TRADING,Bloomberg,https://www.bloomberg.com/news/articles/2014-0...,2014-09-16,Are you a robot?,Why did this happen?\nPlease make sure your br...,Why did this happen?\n\nPlease make sure your ...


In [15]:
df_Bloomberg['date_time']

0    2021-11-18
1    2020-01-02
2    2013-12-04
3    2013-12-17
4    2011-04-07
5    2010-09-02
6    2010-07-02
7    2007-07-13
8    2020-04-07
9    2014-09-16
10   2022-02-28
11   2022-02-28
12   2022-02-28
13   2022-02-28
14   2022-02-28
15   2022-03-01
16   2022-02-28
17   2022-02-28
18   2022-02-28
19   2022-02-28
20   2022-02-28
21   2022-02-28
22   2022-02-28
23   2022-02-28
24   2022-02-28
25   2022-03-01
26   2022-02-28
27   2022-02-28
28   2022-02-28
29   2022-02-28
Name: date_time, dtype: datetime64[ns]

In [16]:
import time
from datetime import datetime, timedelta
todaysdate=datetime.now().strftime("%Y-%m-%d")
# todaysdate=datetime.strptime(str(todaysdate), '%Y-%m-%d')
last_date=(datetime.now() + timedelta(days=-6)).strftime("%Y-%m-%d")
# last_date=datetime.strptime(str(last_date), '%Y-%m-%d')

In [19]:
import pandas as pd
df_Bloomberg['date_time_1']=pd.to_datetime(df_Bloomberg["date_time"]).apply(lambda x: x.date())

In [20]:
df_Bloomberg['date_time_1'] = df_Bloomberg["date_time_1"].apply(lambda x : datetime.strptime(str(x), '%Y-%m-%d') if x!='' else None)

In [21]:
df_Bloomberg['date_time_1']

0    2021-11-18
1    2020-01-02
2    2013-12-04
3    2013-12-17
4    2011-04-07
5    2010-09-02
6    2010-07-02
7    2007-07-13
8    2020-04-07
9    2014-09-16
10   2022-02-28
11   2022-02-28
12   2022-02-28
13   2022-02-28
14   2022-02-28
15   2022-03-01
16   2022-02-28
17   2022-02-28
18   2022-02-28
19   2022-02-28
20   2022-02-28
21   2022-02-28
22   2022-02-28
23   2022-02-28
24   2022-02-28
25   2022-03-01
26   2022-02-28
27   2022-02-28
28   2022-02-28
29   2022-02-28
Name: date_time_1, dtype: datetime64[ns]

In [ ]:
import numpy as np
df_Bloomberg['trigger']=np.where(((df_Bloomberg['date_time_1']>=last_date)
                                 &(df_Bloomberg['date_time_1']<=todaysdate))
                                & (df_Bloomberg['Match_1']>90),'Y','N')